In [1]:
import pandas as pd

## Loading the data

In [2]:
df = pd.read_csv('../data/data.csv')

In [5]:
df.head().T

,0,1,2,3,4
type_school,Academic,Academic,Academic,Vocational,Academic
school_accreditation,A,A,B,B,A
gender,Male,Male,Female,Male,Female
interest,Less Interested,Less Interested,Very Interested,Very Interested,Very Interested
residence,Urban,Urban,Urban,Rural,Urban
parent_age,56,57,50,49,57
parent_salary,6950000,4410000,6500000,6600000,5250000
house_area,83.0,76.8,80.6,78.2,75.1
average_grades,84.09,86.91,87.43,82.12,86.79
parent_was_in_college,False,False,False,True,False


In [6]:
len(df)

1000

In [8]:
df.isnull().sum()

type_school              0
school_accreditation     0
gender                   0
interest                 0
residence                0
parent_age               0
parent_salary            0
house_area               0
average_grades           0
parent_was_in_college    0
in_college               0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [68]:
df_train_all, df_test = train_test_split(df, test_size=0.2, random_state=1)

### EDA

In [69]:
df_train_all.in_college.value_counts()

True     405
False    395
Name: in_college, dtype: int64

In [70]:
y_train_all = df_train_all.in_college.astype(int).values
y_test = df_test.in_college.astype(int).values

In [71]:
y_train_all.mean()

0.50625

In [72]:
categorical = [
    'type_school', 'school_accreditation', 'gender', 'interest', 'residence',
    'parent_was_in_college'
]

In [73]:
df_train_all[categorical].describe()

,type_school,school_accreditation,gender,interest,residence,parent_was_in_college
count,800,800,800,800,800,800
unique,2,2,2,5,2,2
top,Academic,B,Male,Very Interested,Urban,True
freq,488,417,401,262,436,416


In [121]:
for c in categorical:
    print(df_train_all[c].value_counts())
    print()

Academic      488
Vocational    312
Name: type_school, dtype: int64

B    417
A    383
Name: school_accreditation, dtype: int64

Male      401
Female    399
Name: gender, dtype: int64

Very Interested     262
Uncertain           206
Less Interested     185
Quiet Interested     77
Not Interested       70
Name: interest, dtype: int64

Urban    436
Rural    364
Name: residence, dtype: int64

True     416
False    384
Name: parent_was_in_college, dtype: int64



In [74]:
numerical = ['parent_age', 'parent_salary', 'house_area', 'average_grades']

In [75]:
df_train_all[numerical].describe().round(2)

,parent_age,parent_salary,house_area,average_grades
count,800.00,800.00,800.00,800.00
mean,52.24,5372650.00,74.43,86.05
std,3.47,1387610.92,14.95,3.34
min,40.00,1000000.00,20.00,75.00
25%,50.00,4367500.00,64.60,83.72
50%,52.00,5430000.00,75.30,85.50
75%,54.00,6382500.00,84.60,88.19
max,65.00,10000000.00,116.30,98.00


Now let's see which features are important

In [76]:
from sklearn.metrics import mutual_info_score

In [77]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_train_all.in_college)

mi = df_train_all[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

interest                 0.039574
school_accreditation     0.001280
parent_was_in_college    0.001107
residence                0.000351
gender                   0.000313
type_school              0.000205
dtype: float64

It looks like the main categorical factor is the interest of the student

In [78]:
df_train_all[numerical].corrwith(df_train_all.in_college).abs()

parent_age        0.050370
parent_salary     0.481177
house_area        0.443250
average_grades    0.512619
dtype: float64

Parent's salary and house area are probably correlated, and grades seem to be the most contributing factor

In [79]:
df_train_all[['parent_salary']].corrwith(df_train_all.house_area)

parent_salary    0.124039
dtype: float64

It's actually not very correlated - so both should be good features.

Let's train our first model. We'll use all features and then only a subset 

## Training model

In [81]:
df_train, df_val = train_test_split(df_train_all, test_size=0.25, random_state=1)

In [84]:
y_train = df_train.in_college.astype(int).values
y_val = df_val.in_college.astype(int).values

In [85]:
from sklearn.feature_extraction import DictVectorizer

In [87]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [88]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [89]:
from sklearn.linear_model import LogisticRegression

In [90]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [91]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [95]:
y_pred = lr.predict_proba(X_val)[:, 1]

In [96]:
from sklearn.metrics import roc_auc_score

In [97]:
roc_auc_score(y_val, y_pred)

0.7903161264505801

AUC is almost 80% - not bad!

### Pipeline

Let's turn it into a pipeline

In [99]:
from sklearn.pipeline import make_pipeline 

In [100]:
pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression()
)

In [102]:
pipeline.fit(train_dicts, y_train)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('logisticregression', LogisticRegression())])

In [105]:
y_pred = pipeline.predict_proba(val_dicts)[:, 1]

In [106]:
roc_auc_score(y_val, y_pred)

0.7903161264505801

### Saving the model

In [107]:
import pickle

In [112]:
with open('../artifacts/pipeline.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

## Using the model

In [116]:
df_test.iloc[0].to_dict()

{'type_school': 'Academic',
 'school_accreditation': 'A',
 'gender': 'Male',
 'interest': 'Uncertain',
 'residence': 'Rural',
 'parent_age': 48,
 'parent_salary': 7160000,
 'house_area': 71.0,
 'average_grades': 88.46,
 'parent_was_in_college': True,
 'in_college': True}

In [118]:
student = {
    'type_school': 'Academic',
    'school_accreditation': 'A',
    'gender': 'Male',
    'interest': 'Uncertain',
    'residence': 'Rural',
    'parent_age': 48,
    'parent_salary': 7160000,
    'house_area': 71.0,
    'average_grades': 88.46,
    'parent_was_in_college': True,
}

In [120]:
pipeline.predict_proba(student)[0, 1]

0.5812785745673288

In [122]:
import json

In [124]:
print(json.dumps(student, indent=2))

{
  "type_school": "Academic",
  "school_accreditation": "A",
  "gender": "Male",
  "interest": "Uncertain",
  "residence": "Rural",
  "parent_age": 48,
  "parent_salary": 7160000,
  "house_area": 71.0,
  "average_grades": 88.46,
  "parent_was_in_college": true
}
